<a href="https://colab.research.google.com/github/LuisDi98/Audio_a_Lesco/blob/dev/Audio_To_LESCO_models_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install dependencies
! pip install datasets
! pip install transformers
! pip install -U openai-whisper
! pip install git+https://github.com/openai/whisper.git 

# Get Dataset
CIEMPIESS_TEST

In [4]:
#Load the dataset
from datasets import load_dataset, load_metric, Audio
import whisper
ds=load_dataset("ciempiess/ciempiess_test", split="test")

#Downsample to 16kHz
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset ciempiess_test downloaded and prepared to /root/.cache/huggingface/datasets/ciempiess___ciempiess_test/ciempiess_test/1.0.0/6308ec4b2e9db78ec4aa6169117a8b2e2b3ac399c99501e57de918b528539b0b. Subsequent calls will reuse this data.


In [5]:
ds

Dataset({
    features: ['audio_id', 'audio', 'speaker_id', 'gender', 'duration', 'normalized_text'],
    num_rows: 3558
})

In [6]:
ds[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/024bce9383676d0a3a6afc0dd8423b344044040363dd48ff5402220707a9d22c/test/male/M_07/CMPT_M_07_0074.flac',
 'array': array([-0.00192261, -0.00234985, -0.00158691, ..., -0.00839233,
        -0.00900269, -0.00698853]),
 'sampling_rate': 16000}

#Whisper-large
* De momento no ocupamos los inputs, esta es una manera más sencilla.
* Queda pendiente probar wav2vec2-large-xlsr-53-spanish-ep5-944h: https://huggingface.co/carlosdanielhernandezmena/wav2vec2-large-xlsr-53-spanish-ep5-944h
* Comparar con el word error rate


In [7]:
model = whisper.load_model("large")
# Transcribe from audio to text with the first sample
sample = ds[0]["audio"]["path"]
result = model.transcribe(sample)
print("Whisper result: ",result["text"])
print("Normalized_text of the sample: ",ds[0]["normalized_text"])

100%|██████████████████████████████████████| 2.87G/2.87G [00:24<00:00, 125MiB/s]


Whisper result:   Pues ahí está la libertad de las posiciones, de a ver quién es pasivo, quién es activo, bla, bla, bla. Muchas cosas, ¿no? Pero...
Normalized_text of the sample:  pues está la libertá de las posiciones de a ver quién es pasivo quién es activo blablablá muchas cosas no pero


#Whisper-medium

###TO DO: 
Usar el dataset en espaniol y probar el modelo.

Buscar una metrica adecuada y una biblioteca para el testing

Ver porque esta dando samples en ingles y no espaniol al usar el dataset de ciempiess

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-medium")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="spanish", task="transcribe")


# load dummy dataset and read audio files
ds=load_dataset("ciempiess/ciempiess_test", split="test")
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))
sample = ds[0]["audio"]
input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features 
print(input_features)
# generate token ids
predicted_ids = model.generate(input_features)
print(predicted_ids)
# decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)
print(transcription)

transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
print(transcription)


tensor([[[-0.1399,  0.5492,  0.2900,  ..., -0.7534, -0.7534, -0.7534],
         [ 0.2928,  0.3964,  0.2335,  ..., -0.7534, -0.7534, -0.7534],
         [ 0.2252,  0.2252,  0.0790,  ..., -0.7534, -0.7534, -0.7534],
         ...,
         [-0.6674, -0.6745, -0.5624,  ..., -0.7534, -0.7534, -0.7534],
         [-0.5837, -0.6500, -0.5296,  ..., -0.7534, -0.7534, -0.7534],
         [-0.5518, -0.7030, -0.6630,  ..., -0.7534, -0.7534, -0.7534]]])


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[50258, 50259, 50359, 50363,  1042,    11,   456,   307,   264,  5645,
           295,   264,  8432,    11,   295,  2577,   567,   307, 14975,    11,
           567,   307,  4967,    11, 12288, 12288, 12288,    11,   867,   721,
            11,   558,    30,   583,   485, 50257]])
['<|startoftranscript|><|en|><|transcribe|><|notimestamps|> Well, there is the freedom of the positions, of seeing who is passive, who is active, blah blah blah, many things, right? But...<|endoftext|>']
[' Well, there is the freedom of the positions, of seeing who is passive, who is active, blah blah blah, many things, right? But...']


#Whisper-tiny

In [ ]:
model = whisper.load_model("tiny")
# Transcribe from audio to text with the first sample
sample = ds[0]["audio"]["path"]
result = model.transcribe(sample)
print("Whisper result: ",result["text"])
print("Normalized_text of the sample: ",ds[0]["normalized_text"])

100%|██████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 145MiB/s]


Whisper result:   pues esta la libertad de las posiciones de haber quien es pasivo, que en el activo la muchas cosas, ¿no? pero...
Normalized_text of the sample:  pues está la libertá de las posiciones de a ver quién es pasivo quién es activo blablablá muchas cosas no pero


#Haciendo unas pruebas con el voice detection y wav2vec


In [ ]:
!pip install pydub


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import ipywidgets as widgets
from IPython import display as disp
from IPython.display import display, Audio, clear_output
from google.colab import output
import base64
from pydub import AudioSegment
import io
import tempfile
import librosa

In [ ]:
def record_audio(seconds=3,
                 sample_rate=44100,
                 normalize_db=0.1):
    """Record audio from the browser in colab using javascript.
    Based on: https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be
    Args:
      seconds: Number of seconds to record.
      sample_rate: Resample recorded audio to this sample rate.
      normalize_db: Normalize the audio to this many decibels. Set to None to skip
        normalization step.
    Returns:
      An array of the recorded audio at sample_rate.
    """
    # Use Javascript to record audio.
    record_js_code = """
      const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
      const b2text = blob => new Promise(resolve => {
        const reader = new FileReader()
        reader.onloadend = e => resolve(e.srcElement.result)
        reader.readAsDataURL(blob)
      })
      var record = time => new Promise(async resolve => {
        stream = await navigator.mediaDevices.getUserMedia({ audio: true })
        recorder = new MediaRecorder(stream)
        chunks = []
        recorder.ondataavailable = e => chunks.push(e.data)
        recorder.start()
        await sleep(time)
        recorder.onstop = async ()=>{
          blob = new Blob(chunks)
          text = await b2text(blob)
          resolve(text)
        }
        recorder.stop()
      })
      """
    print('Starting recording for {} seconds...'.format(seconds))
    display(disp.Javascript(record_js_code))
    audio_string = output.eval_js('record(%d)' % (seconds * 1000.0))
    print('Finished recording!')
    audio_bytes = base64.b64decode(audio_string.split(',')[1])
    return audio_bytes_to_np(audio_bytes,
                             sample_rate=sample_rate,
                             normalize_db=normalize_db)
    
def audio_bytes_to_np(wav_data,
                      sample_rate=44100,
                      normalize_db=0.1):
    """Convert audio file data (in bytes) into a numpy array.
    Saves to a tempfile and loads with librosa.
    Args:
      wav_data: A byte stream of audio data.
      sample_rate: Resample recorded audio to this sample rate.
      normalize_db: Normalize the audio to this many decibels. Set to None to skip
        normalization step.
    Returns:
      An array of the recorded audio at sample_rate.
    """
    # Parse and normalize the audio.
    audio = AudioSegment.from_file(io.BytesIO(wav_data))
    audio.remove_dc_offset()
    if normalize_db is not None:
        audio.normalize(headroom=normalize_db)
    # Save to tempfile and load with librosa.
    with tempfile.NamedTemporaryFile(suffix='.wav') as temp_wav_file:
        fname = temp_wav_file.name
        audio.export(fname, format='wav')
        audio_np, unused_sr = librosa.load(fname, sr=sample_rate)
    return audio_np

In [ ]:
#@title Record
#@markdown * Set recording time:

SAMPLE_RATE = 44100
record_seconds =   10#@param {type:"number", min:1, max:10, step:1}

def _record_audio(b):
  global audio
  clear_output()
  audio = record_audio(record_seconds, sample_rate=SAMPLE_RATE)
  display(Audio(audio, rate=SAMPLE_RATE))

button = widgets.Button(description="Start recording...")
button.on_click(_record_audio)
display(button)

Starting recording for 10 seconds...


<IPython.core.display.Javascript object>

Finished recording!


# Normalización del texto

Necesitamos normalizar las salidas de whisper, ya que la librería nos proporciona las etiquetas de los audios pero estos están normalizados, este es un paso previo para poder aplicar algunas métricas y que los resultados sean más acertados.

In [31]:
import re
from unicodedata import normalize

def formatText(entryText):
    '''
    Quita las etiquetas xml para extraer el texto, quita caracteres,convierte a minuscula
    y devuelve lista de palabras sin acentos (Como que hace más de una cosa xd)
    '''
    text = re.sub(r'<[^<]+>', "",entryText)
    text = text.lower()
    characters = "#&!?'-[]()\/''=`,:~}{+|" #-> caracteres innecesarios que se quitan 
    text = ''.join(x for x in text if x not in characters)
    return text    

def deleteAccents(text):
    '''
    Borrar acentos del texto, exceptuando ñ
    #Obtenido de https://ideone.com/YcXaQD (ideone.com)
    '''
    #NFD (Normalization Form Canonical Decomposition) expresion para eliminar diacríticos 
    text = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", text), 0, re.I)
    #NFC (#Normalization Form Canonical Composition) volver a su forma compuesta
    text = normalize( 'NFC', text)
    return text

def deleteSpaces(entryText):
    return " ".join(entryText.split())

def normalizeText(entryText):
    newText = formatText(entryText)
    return deleteSpaces(deleteAccents(newText))


In [32]:
normalizeText("No sé, honestamente hice hace mu#cho este código, cuando estaba en RIT & XD! [algo aqui] +, PERO me parece ineficiente")

'no se honestamente hice hace mucho este codigo cuando estaba en rit xd algo aqui pero me parece ineficiente'